In [ ]:
from general import *

In [ ]:
class HairField:
    def __init__(self, parameters):
        self.N_hairs = parameters['N_hairs']
        self.max_joint_angle = parameters['max_joint_angle']
        self.max_list = parameters['max_joint_angle']
        self.min_joint_angle = parameters['min_joint_angle']
        self.min_list = parameters['min_joint_angle']
        self.max_angle = parameters['max_angle']
        self.overlap = parameters['overlap']
        self.receptive_field = None

    def get_receptive_field(self):
        rf = (self.max_joint_angle - self.min_joint_angle) / self.N_hairs
        rf = (1-self.overlap/(self.max_joint_angle - self.min_joint_angle))*rf

        receptive_min = np.linspace(self.min_joint_angle, self.min_joint_angle + rf*(self.N_hairs - 1),
                                    num=self.N_hairs)

        receptive_max = np.linspace(self.max_joint_angle - rf*(self.N_hairs - 1), self.max_joint_angle,
                                    num=self.N_hairs)

        self.receptive_field = np.stack((receptive_min, receptive_max))
        
        
    def get_double_receptive_field(self):
        self.get_receptive_field()

        rf1 = -self.receptive_field.copy() + self.max_joint_angle + self.min_joint_angle

        self.receptive_field = np.hstack((rf1, self.receptive_field))
        self.N_hairs = 2 * self.N_hairs
        
    
    def reset_max_min(self, i):
        self.max_joint_angle = self.max_list[i]
        self.min_joint_angle = self.min_list[i]


    def get_hair_angle(self, x):
        min_rf = self.receptive_field[0, :]
        slope = self.max_angle / (self.receptive_field[1, :] - self.receptive_field[0, :])

        slope, min_rf, x = np.tile(slope, (x.size, 1)), np.tile(min_rf, (x.size, 1)), np.tile(x, (self.N_hairs, 1)).T

        out = np.clip(slope * (x - self.receptive_field[0, :]), 0, 90)

        return out

In [ ]:
parameters_hair_field = {'N_hairs': 10, 'max_joint_angle': 180, 'min_joint_angle': 0, 'max_angle': 90, 'overlap': 4}
joint_angle = np.linspace(-20, 200, num=2000)

hair_field = HairField(parameters_hair_field)
hair_field.get_receptive_field()
hair_angles = hair_field.get_hair_angle(joint_angle)

In [ ]:
fig, ax = plt.subplots()


plt.plot(joint_angle, hair_angles, color=custom_colors[0], linestyle=custom_linestyles[0])

for i in range(hair_field.N_hairs - 1):
    plt.fill_between([hair_field.receptive_field[0, 1+i], hair_field.receptive_field[1, i]], [90, 90], color='grey', alpha=0.35)

ax.set_xlabel("joint angle (°)")
ax.set_ylabel("hair angle (°)")
ax.set_xlim(-10, 190)
ax.minorticks_on()

In [ ]:
fig.tight_layout(pad=parameters['pad'])
fig.savefig('images/afferents/uni_hairplate.png')
fig.savefig('images/afferents/uni_hairplate.pdf')
fig.clear()

In [ ]:
parameters_hair_field = {'N_hairs': 10, 'max_joint_angle': 180, 'min_joint_angle': 0, 'max_angle': 90, 'overlap': 4}

hair_field = HairField(parameters_hair_field)
hair_field.get_receptive_field()
hair_field.get_double_receptive_field()
hair_angles = hair_field.get_hair_angle(joint_angle)

In [ ]:
fig, ax = plt.subplots()

n_2 = int(hair_field.N_hairs/2)
n_4 = int(n_2/2)

plt.plot(joint_angle, hair_angles[:, n_4], color=custom_colors[0], linestyle=custom_linestyles[0], label='Dorsal hair field')
plt.plot(joint_angle, hair_angles[:, 1 + n_4:n_2], color=custom_colors[0], linestyle=custom_linestyles[0])
plt.plot(joint_angle, hair_angles[:, n_2+n_4], color=custom_colors[1], linestyle=custom_linestyles[1], label='Ventral hair field')
plt.plot(joint_angle, hair_angles[:, 1 + n_2+n_4:], color=custom_colors[1], linestyle=custom_linestyles[1])

for i in range(n_4 - 1):
    plt.fill_between([hair_field.receptive_field[0,  n_4 + i + 1], hair_field.receptive_field[1, n_4 + i]], [90, 90], 
                     color=custom_colors[0],
                     alpha=0.25)
    plt.fill_between([hair_field.receptive_field[0, n_2 + n_4 + i + 1], hair_field.receptive_field[1, n_2 + n_4 + i]], [90, 90],
                     color=custom_colors[1],
                     alpha=0.25,)
    
ax.set_xlabel("joint angle (°)")
ax.set_ylabel("hair angle (°)")
#ax.legend(ncol=2, loc=9, bbox_to_anchor=[0.5, 1.15], fancybox=False, edgecolor='black')
ax.set_xlim(-10, 190)
ax.minorticks_on()

In [ ]:
fig.tight_layout(pad=parameters['pad'])
fig.savefig('images/afferents/bi_hairplate_pos.png')
fig.savefig('images/afferents/bi_hairplate_pos.pdf')
fig.clear()

In [ ]:
fig, ax = plt.subplots()

plt.plot(joint_angle, hair_angles[:, n_2], color=custom_colors[0], linestyle=custom_linestyles[0], label='Dorsal hair field')
plt.plot(joint_angle, hair_angles[:, n_2], color=custom_colors[1], linestyle=custom_linestyles[1], label='Ventral hair field')

plt.plot(joint_angle, hair_angles[:, :n_2], color=custom_colors[0], linestyle=custom_linestyles[0])
plt.plot(joint_angle, hair_angles[:, n_2:], color=custom_colors[1], linestyle=custom_linestyles[1])
    
ax.set_xlabel("joint angle (°)")
ax.set_ylabel("hair angle (°)")
#ax.legend(ncol=2, loc=9, bbox_to_anchor=[0.5, 1.15], fancybox=False, edgecolor='black')
ax.set_xlim(-10, 190)
ax.minorticks_on()

In [ ]:
fig.tight_layout(pad=parameters['pad'])
fig.savefig('images/afferents/bi_hairplate_vel.png')
fig.savefig('images/afferents/bi_hairplate_vel.pdf')
fig.clear()

In [ ]:
class SensoryNeuron:
    NeuronState = namedtuple('NeuronState', ['V', 'w', 'spk'])

    def __init__(self, parameters):
        super(SensoryNeuron, self).__init__()
        self.C = parameters['C']
        self.g_L = parameters['g_L']
        self.E_L = parameters['E_L']
        self.DeltaT = parameters['DeltaT']
        self.a = parameters['a']
        self.V_T = parameters['V_T']
        self.tau_W = parameters['tau_W']
        self.b = parameters['b']
        self.V_R = parameters['V_R']
        self.V_cut = parameters['V_cut']
        self.n = parameters['n']
        self.dt = parameters['dt']
        self.state = None

    def initialize_state(self):
        self.state = None

    def forward(self, input):
        if self.state is None:
            self.state = self.NeuronState(V=np.linspace(self.E_L, self.E_L+10e-3, self.n),
                                          w=np.zeros(self.n),
                                          spk=np.zeros(self.n))
        V = self.state.V
        w = self.state.w
        I = input
        
        V += (self.g_L * (self.E_L - V) + self.g_L * self.DeltaT * np.exp(
            (V - self.V_T) / self.DeltaT) - w + I) * self.dt / self.C

        spk = np.heaviside(V - self.V_cut, 0)
        
        V = (1 - spk) * V + spk * self.V_R
        
        w += spk * self.b
        w += (self.a * (V - self.E_L) - w) * self.dt / self.tau_W

        self.state = self.NeuronState(V=V, w=w, spk=spk)

        return V, spk

In [ ]:
def ramp_generator(n_ramp, n_steps, height, base=0):
    n_dims = height.size
    
    input_ramp = np.zeros((n_steps, n_dims))

    for i in range(n_dims):
        increase = np.linspace(base, height[i], n_ramp[i])
        constant = np.linspace(height[i], height[i], n_steps - 2 * n_ramp[i])
        decrease = np.linspace(height[i], base, n_ramp[i])
        
        input_ramp[:, i] = np.concatenate((increase, constant, decrease))
        
    return input_ramp

In [ ]:
sensory_parameters = {'C': 200e-12, 'g_L': 2e-9, 'E_L': -70e-3, 'DeltaT': 2e-3, 'a': 2e-9, 'V_T': -50e-3,
                      'tau_W': 50e-3, 'b': 264e-12, 'V_R': -70e-3, 'V_cut': -40e-3, 'n': 5, 'dt': 0.0001}

sensory_neuron = SensoryNeuron(sensory_parameters)
sensory_neuron.initialize_state()

In [ ]:
v_var, v_stat = np.array([24, 47, 88, 151, 245])*4, 60*4
max_angle_var, max_angle_stat = np.array([15, 23, 34, 46, 60]), 37
t_total = 3
j=0

N_steps = round(t_total/sensory_parameters['dt'])
spike_train, input_ramp = np.empty((N_steps + 2, v_var.size, 2)), np.empty((N_steps, v_var.size, 2))
for [v, max_angle] in [[v_var, max_angle_stat], [v_stat, max_angle_var]]:

    N_ramp = np.around(max_angle/(v*sensory_parameters['dt'])).astype(int)
    height = np.empty(sensory_parameters['n'])
    height[:] = max_angle/18e9
    
    input_ramp[:, :, j] = ramp_generator(N_ramp, N_steps, height)

    sensory_neuron.initialize_state()
    
    time = np.array([])
    for i in range(N_steps):
        _, spike_train[i, :, j] = sensory_neuron.forward(input_ramp[i, :, j])
        time = np.append(time, i*sensory_parameters['dt'])
        
    j +=1

In [ ]:
fig, [ax, ax2] = plt.subplots(2, figsize=(3.75*0.95, 3.75*0.95), gridspec_kw={'height_ratios': [1, 2]}, sharex='all')
plt.subplots_adjust(wspace=0, hspace=0.05)

for i in range(v_var.size):
    firing_rate, spike_index = get_firing_rate_ISI(spike_train[:, i, 1], sensory_parameters['dt'])
    ax.plot(time, input_ramp[:, i, 1]*18e9, color=custom_colors[i], linestyle=custom_linestyles[i])
    ax2.plot(time[spike_index], firing_rate, color=custom_colors[i], linestyle=custom_linestyles[i])

ax.set_ylabel("Hair angle (°)")
ax.grid(axis='y')

ax2.set_xlabel("Time (s)")
ax2.set_ylabel("Spike rate (Hz)")
ax.set_ylim([0, 65])
ax.set_yticks([0, 20, 40, 60])
ax2.grid(axis='y')
ax2.legend(['θ = 15°', 'θ = 23°', 'θ = 34°', 'θ = 46°', 'θ = 60°'], fancybox=False, edgecolor='black')  
ax2.set_yticks([0, 40, 80, 120, 160, 200, 240, 280])
ax2.minorticks_on()

In [ ]:
fig.tight_layout(pad=parameters['pad'])
fig.savefig('images/afferents/single_hair_constant_angle.png')
fig.savefig('images/afferents/single_hair_constant_angle.pdf')

In [ ]:
fig, [ax, ax2] = plt.subplots(2, figsize=(3.75*0.95, 3.75*0.95), gridspec_kw={'height_ratios': [1, 2]}, sharex='all')
plt.subplots_adjust(wspace=0, hspace=0.05)

for i in range(5):
    firing_rate, spike_index = get_firing_rate_ISI(spike_train[:, i, 0], sensory_parameters['dt'])
    ax.plot(time, input_ramp[:, i, 0]*18e9, color=custom_colors[i], linestyle=custom_linestyles[i])
    ax2.plot(time[spike_index], firing_rate, color=custom_colors[i], linestyle=custom_linestyles[i])

ax.set_ylabel("Hair angle (°)")
ax.grid(axis='y')
ax.set_yticks([0, 10, 20, 30, 40])

ax2.set_xlabel("Time (s)")
ax2.set_ylabel("Spike rate (Hz)")
ax2.grid(axis='y')
ax.set_ylim([0, 65])
ax.set_yticks([0, 20, 40, 60])
ax2.legend(['ω = 96 °/s', 'ω = 188 °/s', 'ω = 352 °/s', 'ω = 604 °/s', 'ω = 980 °/s'], fancybox=False, edgecolor='black', loc='lower center')
ax2.set_yticks([0, 40, 80, 120, 160])
ax2.minorticks_on()

In [ ]:
fig.tight_layout(pad=parameters['pad'])
fig.savefig('images/afferents/single_hair_variable_angle.png')
fig.savefig('images/afferents/single_hair_variable_angle.pdf')

In [ ]:
with open('data/joint_angles_' + parameters['joint_string'], 'rb') as file:
    joint_angles = pickle.load(file)
    
 
sensory_parameters['n'] = parameters['N_HAIRS']*36
sensory_parameters['dt'] = constants['dt']
sensory_neuron = SensoryNeuron(sensory_parameters)
spike_sensory = np.empty((constants['N_STEPS'], constants['N_ANGLES'] * parameters['N_HAIRS'] * 2, parameters['N_SIMULATIONS']), dtype=np.uint8)

time = np.linspace(0, constants['T_TOTAL'], num=constants['N_STEPS'])

In [ ]:
for k in tqdm(range(parameters['N_SIMULATIONS'])):
    
    joint_angle = joint_angles[:, :, k]

    hair_angles = np.empty((joint_angle.shape[0], 2 * joint_angle.shape[1] * parameters['N_HAIRS']), dtype=np.float32)
    hair_field_parameters = {'N_hairs': parameters['N_HAIRS'], 'min_joint_angle': np.amin(joint_angle, axis=0), 
                             'max_joint_angle': np.amax(joint_angle, axis=0), 'max_angle': 90, 'overlap': 0.1}
    
    for i in range(constants['N_ANGLES']):
        hair_field = HairField(hair_field_parameters)
        hair_field.reset_max_min(i)
        hair_field.get_double_receptive_field()
        hair_angles[:, i * 2 * parameters['N_HAIRS']: 2 * parameters['N_HAIRS'] + i * 2 * parameters['N_HAIRS']] = (
                hair_field.get_hair_angle(joint_angle[:, i]) / 18e9)
    
    sensory_neuron.initialize_state()
        
    for i in range(constants['N_STEPS']):
        _, spike_sensory[i, :, k] = sensory_neuron.forward(hair_angles[i, :])

In [ ]:
with open('data/spike_sensory', 'wb') as file:
        np.save(file, spike_sensory)

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

N_half = parameters['N_HAIRS'] // 2
diff = hair_field.max_list[0] - hair_field.min_list[0]

spike_sensory = spike_sensory[:, :, 0].astype(np.float32)
spike_sensory[spike_sensory == 0] = np.nan

i = 0 
ax1.scatter(time, (-i + N_half) * spike_sensory[:, i + N_half], color=custom_colors[0], s=0.1, label = 'Spike - domain')
ax1.scatter(time, (1 + i + N_half) * spike_sensory[:, i + N_half + parameters['N_HAIRS']], color=custom_colors[1], s=0.1, label = 'Spike + domain')

for i in range(N_half):
    ax1.scatter(time, (-i + N_half) * spike_sensory[:, i + N_half], color=custom_colors[0], s=0.1, label = '')
    ax1.scatter(time, (1 + i + N_half) * spike_sensory[:, i + N_half + parameters['N_HAIRS']], color=custom_colors[1], s=0.1)

ax2.plot(time, joint_angles[:, 0, 0], color='black', label = 'Exp. data')
ax1.plot(time, np.full(time.shape, N_half + 0.5), linestyle='dotted', color='black', label = 'Resting angle')
ax1.set_ylim(1 - parameters['N_HAIRS'] * .05, parameters['N_HAIRS'] * 1.05)
ax2.set_ylim(joint_angles[:, 0, 0].min() - .05 * diff, joint_angles[:, 0, 0].max() + .05 * diff)

ax1.set_xlabel("Time (s)")
ax1.set_ylabel("Neuron index")
ax1.set_yticks(np.arange(1, parameters['N_HAIRS'] + 2)[::5])
ax2.set_ylabel("Joint angle (°)")

ax2.minorticks_on()
ax1.minorticks_on()
#ax1.set_ylim([1, n_index+1])
fig.legend(fancybox=False, edgecolor='black', bbox_to_anchor=[0.88, 0.54])

fig.tight_layout(pad=parameters['pad'])
fig.savefig('images/afferents/spike_timing_.png')
fig.savefig('images/afferents/spike_timing_.pdf')
fig.show()

In [ ]:
fig, [ax, ax2, ax3, ax4] = plt.subplots(4, figsize=(5.728, 1.5*3.54), gridspec_kw={'height_ratios': [1, 2.5, 1, 1]}, sharex='all')
plt.subplots_adjust(wspace=0, hspace=0.05)

sensory_parameters = {'C': 200e-12, 'g_L': 2e-9, 'E_L': -70e-3, 'DeltaT': 2e-3, 'a': 2e-9, 'V_T': -50e-3,
                      'tau_W': 100e-3, 'b': 0.1e-9, 'V_R': -70e-3, 'V_cut': -40e-3, 'n': 1, 'dt': 0.0001}

sensory_neuron = SensoryNeuron(sensory_parameters)
sensory_neuron.initialize_state()

input = np.zeros(2000)
input[50:] = 1e-9

voltage, spikes, time, omega = np.zeros(input.size), np.zeros(input.size), np.zeros(input.size), np.zeros(input.size)

for i in range(input.size):
    voltage[i], spikes[i] = sensory_neuron.forward(input[i])
    omega[i] = sensory_neuron.state.w
    time[i] = i*0.0001 
    
ax.plot(time, input*1e9, color=custom_colors[0])    

ax.set_ylabel("I (nA)")
ax.set_yticks([0, 1])
ax.set_xticks([0, 0.1, 0.2])
ax.set_ylim(0, 1.5)

ax2.plot(time, voltage*1000, color=custom_colors[0])

ax2.set_ylabel("Voltage (mV)")
ax2.set_yticks([-70, -50])

ax3.plot(time, omega*1e9, color=custom_colors[0])

ax3.set_ylabel("ω (nA)")
#ax3.set_ylim(0, 1.5)

ax4.plot(time, spikes, color=custom_colors[0])

ax4.set_ylabel("post spk")
ax4.set_yticks([0, 1])
ax4.set_xlabel("Time (s)")
ax4.set_ylim(0, 1.5)
               
fig.tight_layout(pad=parameters['pad'])
fig.savefig('images/afferents/AdEx_dynamics.png')
fig.savefig('images/afferents/AdEx_dynamics.pdf')